# Simple Version 40

This notebook is an attempt to create a very simple version of the finite algebra code.

In [204]:
import numpy as np

class FiniteAlgebra:

    def __init__(self, name, description, elements, table):

        self._name = name
        self._desc = description

        if len(elements) != len(set(elements)):
            raise ValueError(f"Repeated elements are not permitted")
        self._elems = elements
        
        # If necessary, convert table of str into a table of int
        elem00 = table[0][0]
        if isinstance(elem00, str):
            _tbl = [[elements.index(elem) for elem in row] for row in table]
        elif isinstance(elem00, int):
            _tbl = table
        else:
            raise TypeError("Table entries must be all str or all int.")

        self._table = np.array(_tbl, dtype=int)
        nrows, ncols = self._table.shape
        if nrows != ncols:
            raise ValueError(f"The table is not square.")
        if nrows != len(elements):
            raise ValueError(f"Number of elements must equal number of rows/cols.")

        self._inv_lookup = None
        if self.has_inverses:
            # Create a dictionary that maps each of the algebra's elements to its inverse element.
            row_indices, col_indices = np.where(self._table == self.elements.index(self.identity))
            self._inv_lookup = {self.elements[elem_index]: self.elements[elem_inv_index]
                                for (elem_index, elem_inv_index)
                                in zip(row_indices, col_indices)}

    @property
    def name(self):
        return self._name

    @property
    def description(self):
        return self._desc

    def __repr__(self):
        return f"{self.__class__.__name__}({self._name!r}, {self._desc!r}, {self._elems}, {self._table.tolist()})"

    def __getitem__(self, index: int) -> str:
        """Returns the algebra's element at position index."""
        return self._elems[index]

    def __len__(self):
        return len(self._elems)
        
    def __call__(self, elem1: str, elem2: str) -> str:
        """Returns elem1 * elem2, according to the algebra's table"""
        row = self._elems.index(elem1)
        col = self._elems.index(elem2)
        index = self._table[row, col]
        return self._elems[index]

    @property
    def elements(self):
        return self._elems

    def inv(self, elem):
        return self._inv_lookup[elem]

    def table(self, as_indices=True):
        if as_indices:
            return self._table.tolist()
        else:
            return [[self._elems[index] for index in row] for row in self._table.tolist()]

    def element_names(self, indices):
        """A convenience method for turning a set of element indices into their resp. names.
        Used below to display counterexamples, when requested."""
        return [self._elems[index] for index in indices]

    def is_associative(self, show_counter_example=False) -> bool:
        """Returns True if the algebra is associative, otherwise returns False"""
        indices = range(len(self))
        result = True
        counter_example = None
        for a in indices:
            for b in indices:
                for c in indices:
                    ab = self._table[a][b]
                    bc = self._table[b][c]
                    if not (self._table[ab][c] == self._table[a][bc]):
                        if show_counter_example:
                            print("A counter example to associativity:")
                            _a, _b, _c = [self._elems[index] for index in [a, b, c]]
                            print(f"  ({_a} * {_b}) * {_c} = {self._elems[self._table[ab][c]]}")
                            print(f"  {_a} * ({_b} * {_c}) = {self._elems[self._table[a][bc]]}")
                        result = False
                        return result
        return result

    def is_commutative(self, show_counter_example=False) -> bool:
        """Returns True if the algebra is commutative, otherwise returns False"""
        n = len(self)
        result = True
        for a in range(n):
            # Loop over the table's upper off-diagonal elements
            for b in range(a + 1, n):
                if self._table[a][b] != self._table[b][a]:
                    if show_counter_example:
                        print("A counter example to commutativity:")
                        _a, _b = [self._elems[index] for index in [a, b]]
                        print(f"  {_a} * {_b} = {self._elems[self._table[a][b]]}")
                        print(f"  {_b} * {_a} = {self._elems[self._table[b][a]]}")
                    result = False
                    return result
        return result

    @property
    def left_identity(self):
        """Returns the table's left identity element, if it exists, otherwise None is returned."""
        indices = range(len(self))
        lid = None
        for x in indices:
            if all(self._table[x][y] == y for y in indices):
                lid = x
                break
        if lid is not None:
            return self._elems[lid]
        else:
            return None

    @property
    def right_identity(self):
        """Returns the table's right identity element, if it exists, otherwise None is returned."""
        indices = range(len(self))
        rid = None
        for x in indices:
            if all(self._table[y][x] == y for y in indices):
                rid = x
                break
        if rid is not None:
            return self._elems[rid]
        else:
            return None

    @property
    def identity(self):
        """Returns the table's identity element, if it exists, otherwise None is returned."""
        left_id = self.left_identity
        right_id = self.right_identity
        if (left_id is not None) and (right_id is not None):
            # If both left and right identities exist, then they are necessarily equal.
            return left_id
        else:
            return None

    @property
    def has_inverses(self):
        """Returns True or False, depending on whether the table supports inverses for
        all elements."""
        if self.identity is not None:
            row_indices, col_indices = np.where(self._table == self._elems.index(self.identity))
            if set(row_indices) == set(col_indices):
                if len(row_indices) == len(self):
                    return True
                else:
                    return False
            else:
                return False
        else:
            return False

        # Create a dictionary that maps each of the algebra's elements to its inverse element.
        row_indices, col_indices = np.where(self._table == self.elements.index(alg.identity))
        return {self.elements[elem_index]: self.elements[elem_inv_index]
                for (elem_index, elem_inv_index)
                in zip(row_indices, col_indices)}

In [205]:
rps_elements = ['r', 'p', 's']

rps_table =[['r', 'p', 'r'],
            ['p', 'p', 's'],
            ['r', 's', 's']]

In [206]:
rps = FiniteAlgebra("RPS", "Rock-Paper-Scissors", rps_elements, rps_table)
rps

FiniteAlgebra('RPS', 'Rock-Paper-Scissors', ['r', 'p', 's'], [[0, 1, 0], [1, 1, 2], [0, 2, 2]])

In [207]:
FiniteAlgebra('RPS', 'Rock-Paper-Scissors', ['r', 'p', 's'], [[0, 1, 0], [1, 1, 2], [0, 2, 2]])

FiniteAlgebra('RPS', 'Rock-Paper-Scissors', ['r', 'p', 's'], [[0, 1, 0], [1, 1, 2], [0, 2, 2]])

In [208]:
rps.table()

[[0, 1, 0], [1, 1, 2], [0, 2, 2]]

In [209]:
rps.table(as_indices=False)

[['r', 'p', 'r'], ['p', 'p', 's'], ['r', 's', 's']]

In [210]:
rps[0]  # Return first element

'r'

In [211]:
rps('r', 'p')

'p'

In [212]:
rps.is_associative()

False

In [213]:
rps.is_commutative()

True

In [214]:
try:
    FiniteAlgebra("T0", "Too many elements", ['r', 'p', 's', 'x'], rps_table)
except Exception as msg:
    print(msg)

Number of elements must equal number of rows/cols.


In [215]:
try:
    FiniteAlgebra("T1", "Repeated elements", ['r', 'p', 's', 's'], rps_table)
except Exception as msg:
    print(msg)

Repeated elements are not permitted


In [216]:
test_table_1 = [['r', 'p', 'r'],
                ['p', 'p', 's'],
                ['r', 's', 's'],
                ['s', 'p', 'r']]

try:
    FiniteAlgebra("T2", "Table not square", ['r', 'p', 's'], test_table_1)
except Exception as msg:
    print(msg)

The table is not square.


In [217]:
try:
    FiniteAlgebra("T3", "Table not square", ['r', 'p'], rps_table)
except Exception as msg:
    print(msg)

's' is not in list


In [218]:
test_table_2 = [['r', 'p', 'r'],
                ['p', 'p', 'r'],
                ['r', 'r', 'p']]

try:
    FiniteAlgebra("T3", "No 's' in table", ['r', 'p', 's'], test_table_2)
except Exception as msg:
    print(msg)

In [219]:
if rps.identity is None:
    print("No identity element")
else:
    print(rps.identity)

No identity element


In [220]:
z3 = FiniteAlgebra('Z3',
                   'Cyclic group of order 3',
                   ['e', 'a', 'a^2'],
                   [[ 'e' ,  'a' , 'a^2'],
                    [ 'a' , 'a^2',  'e' ],
                    ['a^2',  'e' ,  'a' ]])
z3

FiniteAlgebra('Z3', 'Cyclic group of order 3', ['e', 'a', 'a^2'], [[0, 1, 2], [1, 2, 0], [2, 0, 1]])

In [221]:
z3('a', 'a^2')

'e'

In [222]:
z3.identity

'e'

In [223]:
z3.has_inverses

True

In [224]:
rps.has_inverses

False

In [225]:
z3.elements

['e', 'a', 'a^2']

In [226]:
[z3.inv(x) for x in z3.elements]

['e', 'a^2', 'a']

In [233]:
class Magma(FiniteAlgebra):

    def __init__(self, alg):
        super().__init__(alg.name, alg.description, alg.elements, alg.table())

In [234]:
# mag = Magma("RPS", "Rock-Paper-Scissors", rps_elements, rps_table)
mag = Magma(rps)

In [229]:
mag

Magma('RPS', 'Rock-Paper-Scissors', ['r', 'p', 's'], [[0, 1, 0], [1, 1, 2], [0, 2, 2]])